In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os, sys

In [32]:
cd ..

/home/jovyan/ASTR_136


In [33]:
hdu_1 = fits.open("BIAS/Stuff/Project1_SB.00000002.BIAS.fits")
hdu_2 = fits.open("BIAS/Stuff/Project1_SB.00000003.BIAS.fits")

In [34]:
x = hdu_1[0].data
y = hdu_2[0].data
print(x.shape)
print(y.shape)

(510, 765)
(510, 765)


In [36]:
cd ASTR_136

[Errno 2] No such file or directory: 'ASTR_136'
/home/jovyan/ASTR_136


In [37]:
files = os.listdir('BIAS/Stuff')
biasdata = np.zeros((510, 765, len(files)))

In [38]:
cd BIAS/Stuff

/home/jovyan/ASTR_136/BIAS/Stuff


In [39]:
for i in range(len(files)):
    hdu = fits.open(files[i])
    biasdata[:,:, i] = hdu[0].data

In [40]:
biasdata[0].shape

(765, 20)

In [41]:
bias_master_mean = np.mean(biasdata, axis =2)
bias_master_median = np.median(biasdata, axis =2)

In [63]:
#plt.imshow(bias_master_mean)
#plt.show()
#plt.imshow(bias_master_mean, vmin= np.percentile(bias_master_mean, 16), vmax =np.percentile(bias_master_mean, 84))
#plt.show()

In [43]:
pwd

'/home/jovyan/ASTR_136/BIAS/Stuff'

In [44]:
cd ../..


/home/jovyan/ASTR_136


In [45]:
files_dark = os.listdir('DARK/')
dark_data = np.zeros((510, 765, len(files_dark)))

In [46]:
cd DARK

/home/jovyan/ASTR_136/DARK


In [47]:
exp_time = []
for i in range(len(files_dark)):
    hdu_dark = fits.open(files_dark[i])
    dark_data[:,:, i] = hdu_dark[0].data #retrieving data from each individual fits file
    exp_times = hdu_dark[0].header['EXPTIME']
    exp_time.append(exp_times)

In [48]:
exp_time= np.array(exp_time)
print((exp_time))

[6. 6. 6. 3. 6. 6. 6. 6. 6. 3. 3. 3. 3. 6. 1. 2. 1. 2. 2. 3. 3. 6. 2. 1.
 2. 1. 1. 3. 1. 2. 2. 3. 1. 1. 1. 2. 2. 1. 3. 2.]


In [49]:
bias_master = bias_master_mean[:,:,None]
z = np.arange(1,41,1)
for i in z:
    dark_reduced = dark_data - bias_master #dark_master = individ dark fits subtracted by bias

In [50]:
dark_master = dark_reduced/exp_time

In [51]:
cd ..

/home/jovyan/ASTR_136


In [52]:
files_flat = [f for f in os.listdir('TWI/') if f.endswith('.fits')]
flat_data = np.zeros((510, 765, len(files_flat)))

In [53]:
cd TWI

/home/jovyan/ASTR_136/TWI


In [54]:
exp_time_flat = []
times = []
for i in range(len(files_flat)):
    hdu_flat = fits.open(files_flat[i])
    flat_data[:,:, i] = hdu_flat[0].data #retrieving data from each individual fits file
    exp_times_flat = hdu_flat[0].header['EXPTIME']
    time= hdu_flat[0].header['TIME-OBS'].split(':')
    times.append(time)
    exp_time_flat.append(exp_times_flat)
    exp_flat = np.array(exp_time_flat)

In [55]:
dark_mean = np.mean(dark_master, axis=2, keepdims=True)

In [56]:
flat_master = ((flat_data - bias_master)/exp_flat)-dark_mean #here it's still all 67 images

In [57]:
flat_mean = np.mean(flat_master, axis=2, keepdims=True) #collapses shape into (510,765,1)

In [58]:
science_image = flat_master/flat_mean

In [59]:
science_median = np.median(science_image, axis=2, keepdims=True) #collapses shape into (510,765,1)

In [60]:
times = np.array(times)

In [62]:
times

array([['02', '55', '29.372'],
       ['02', '58', '55.727'],
       ['03', '02', '02.814'],
       ['02', '58', '38.879'],
       ['02', '55', '35.799'],
       ['03', '05', '33.765'],
       ['02', '58', '51.515'],
       ['03', '04', '56.309'],
       ['03', '05', '11.332'],
       ['03', '01', '40.911'],
       ['02', '55', '38.982'],
       ['02', '58', '43.076'],
       ['03', '11', '45.180'],
       ['03', '04', '48.759'],
       ['03', '05', '41.222'],
       ['02', '58', '26.212'],
       ['03', '17', '45.420'],
       ['03', '20', '39.579'],
       ['03', '18', '47.976'],
       ['03', '11', '00.174'],
       ['03', '15', '40.417'],
       ['03', '09', '45.590'],
       ['03', '23', '42.084'],
       ['03', '10', '15.152'],
       ['03', '10', '37.694'],
       ['03', '12', '52.650'],
       ['03', '13', '15.286'],
       ['03', '13', '37.765'],
       ['03', '12', '30.170'],
       ['03', '08', '55.561'],
       ['03', '14', '37.970'],
       ['03', '01', '51.894'],
       [

In [64]:
science_median.shape

(510, 765, 1)

In [ ]:
plt.xlabel('nth nearest neighbors') 
plt.ylabel('correlation coefficients') 
plt.title('optimal n value')
ax = plt.subplot(111)
plt.plot(times, science_median, label="projected" , color = 'darkblue')
plt.show() 

In [ ]:
#make flat_master by following same protocol but using TWI SKI
# make final flat image by subtracting flat_master - dark_master

In [ ]:
#do a best fit line and return goodness of fit
#times vs. sky brightess (time = min)
#